## HW3

1. обучить несколько разных моделей на наборе данных ССЗ (train_case2.csv): логрег, бустинг, лес и т.д - на ваш выбор 2-3 варианта
2. при обучении моделей обязательно использовать кроссвалидацию

In [101]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion

from sklearn.metrics import precision_recall_curve, roc_curve, roc_auc_score
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [102]:
df = pd.read_csv('train_case2.csv', ';')
df.head(3)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_16308\2324205212.py:1: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  df = pd.read_csv('train_case2.csv', ';')


,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1


In [103]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns='cardio'), df['cardio'], random_state = 26)

In [104]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X[[self.key]]

class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []
        
    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self
    
    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]

**Обнаружилось, что показатель 'gluc' состоит из 3 категорий. Так что в отличие от ноутбука с урока я отнесла этот показатель к категорийным**

In [105]:
continuos_cols = ['age', 'height', 'weight', 'ap_hi', 'ap_lo']
cat_cols = ['gender', 'cholesterol', 'gluc']
base_cols = ['smoke', 'alco', 'active']

continuos_transformers = []
cat_transformers = []
base_transformers = []

In [106]:
for cont_col in continuos_cols:
    transformer = Pipeline([
        ('selector', NumberSelector(key=cont_col)),
        ('standart', StandardScaler())
    ])
    continuos_transformers.append((cont_col, transformer))

for cat_col in cat_cols:
    cat_transformer = Pipeline([
        ('selector', ColumnSelector(key=cat_col)),
        ('ohe', OHEEncoder(key=cat_col))
    ])
    cat_transformers.append((cat_col, cat_transformer))
    
for base_col in base_cols:
    base_transformer = Pipeline([
        ('selector', NumberSelector(key=cont_col))
    ])
    base_transformers.append((base_col, base_transformer))

In [107]:
feats = FeatureUnion(continuos_transformers+cat_transformers+base_transformers)
feature_processing = Pipeline([('feats', feats)])
feature_processing.fit_transform(X_train)

array([[ 1.53098947e+00, -1.64178219e-01, -1.48714440e-02, ...,
         9.00000000e+01,  9.00000000e+01,  9.00000000e+01],
       [-1.40325068e-01,  2.03249163e-01, -5.70561522e-01, ...,
         8.00000000e+01,  8.00000000e+01,  8.00000000e+01],
       [-1.74673419e+00,  6.93152339e-01, -6.40022781e-01, ...,
         8.00000000e+01,  8.00000000e+01,  8.00000000e+01],
       ...,
       [-1.44857492e+00, -1.02150878e+00, -3.62177742e-01, ...,
         8.00000000e+01,  8.00000000e+01,  8.00000000e+01],
       [-2.13343664e-01,  1.30553131e+00,  8.18663672e-01, ...,
         8.00000000e+01,  8.00000000e+01,  8.00000000e+01],
       [-4.49437126e-01, -1.64178219e-01, -5.70561522e-01, ...,
         9.00000000e+01,  9.00000000e+01,  9.00000000e+01]])

In [108]:
classifier_LogReg = Pipeline([
    ('features', feats),
    ('classifier', LogisticRegression(random_state=26, max_iter = 500)),
])

classifier_RandFor = Pipeline([
    ('features', feats),
    ('classifier', RandomForestClassifier(random_state=26, min_samples_leaf=5)),
])

classifier_GradBoost = Pipeline([
    ('features', feats),
    ('classifier', GradientBoostingClassifier(random_state=26, min_samples_leaf=5)),
])

**Т.к. классы несбалансированы и мы оцениваем все-таки наличие заболеваний, будем оценивать по полноте (лучше мы посчитаем здорового человека за больного, чем упустим больного)**

In [109]:
# Cross-validation
cv_scores_LogReg = cross_val_score(classifier_LogReg, X_train, y_train, cv=16, scoring='recall')
cv_score_LogReg = np.mean(cv_scores_LogReg)
cv_score_std_LogReg = np.std(cv_scores_LogReg)
print(f'LogReg CV score is {cv_score_LogReg}+-{cv_score_std_LogReg}')

LogReg CV score is 0.6721258601982729+-0.008670662556912795


In [110]:
cv_scores_RandFor = cross_val_score(classifier_RandFor, X_train, y_train, cv=16, scoring='recall')
cv_score_RandFor = np.mean(cv_scores_RandFor)
cv_score_std_RandFor = np.std(cv_scores_RandFor)
print(f'RandFor CV score is {cv_score_RandFor}+-{cv_score_std_RandFor}')

RandFor CV score is 0.6918183625392013+-0.010371420154928673


In [111]:
cv_scores_GradBoost = cross_val_score(classifier_GradBoost, X_train, y_train, cv=16, scoring='recall')
cv_score_GradBoost = np.mean(cv_scores_GradBoost)
cv_score_std_GradBoost = np.std(cv_scores_GradBoost)
print(f'GradBoostCV score is {cv_score_GradBoost}+-{cv_score_std_GradBoost}')

GradBoostCV score is 0.6969602766754358+-0.010319119467121773


In [112]:
# classifier_LogReg.fit(X_train, y_train)
# y_pred_LogReg = classifier_LogReg.predict_proba(X_test)[:,1]

classifier_RandFor.fit(X_train, y_train)
y_pred_RandFor = classifier_RandFor.predict_proba(X_test)[:,1]

# classifier_GradBoost.fit(X_train, y_train)
# y_pred_GradBoost = classifier_GradBoost.predict_proba(X_test)[:,1]

**По причине, указанной выше, придадим больше веса полноте и поставим b = 2**

In [113]:
b=2
prec_LogReg, rec_LogReg, thre_LogReg = precision_recall_curve(y_test.values, y_pred_LogReg)
f_LogReg = (1+b**2)*(prec_LogReg * rec_LogReg) / (b**2*prec_LogReg + rec_LogReg)
ix_lr = np.argmax(f_LogReg)
rocauc_LogReg =  round(roc_auc_score(y_test.values, y_pred_LogReg),3)
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thre_LogReg[ix_lr], 
                                                                        f_LogReg[ix_lr],
                                                                        prec_LogReg[ix_lr],
                                                                        rec_LogReg[ix_lr]))

Best Threshold=0.234498, F-Score=0.838, Precision=0.544, Recall=0.969


In [114]:
b=2
prec_RandFor, rec_RandFor, thre_RandFor = precision_recall_curve(y_test.values, y_pred_RandFor)
f_RandFor = (1+b**2)*(prec_RandFor * rec_RandFor) / (b**2*prec_RandFor + rec_RandFor)
ix_rf = np.argmax(f_RandFor)
rocauc_RandFor =  round(roc_auc_score(y_test.values, y_pred_RandFor),3)
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thre_RandFor[ix_rf], 
                                                                        f_RandFor[ix_rf],
                                                                        prec_RandFor[ix_rf],
                                                                        rec_RandFor[ix_rf]))

Best Threshold=0.154893, F-Score=0.841, Precision=0.536, Recall=0.979


In [115]:
ix_rf

1567

In [116]:
b=2
prec_GradBoost, rec_GradBoost, thre_GradBoost = precision_recall_curve(y_test.values, y_pred_GradBoost)
f_GradBoost = (1+b**2)*(prec_GradBoost * rec_GradBoost) / (b**2*prec_GradBoost + rec_GradBoost)
ix_gb = np.argmax(f_GradBoost)
rocauc_GradBoost =  round(roc_auc_score(y_test.values, y_pred_GradBoost),3)
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thre_GradBoost[ix_gb], 
                                                                        f_GradBoost[ix_gb],
                                                                        prec_GradBoost[ix_gb],
                                                                        rec_GradBoost[ix_gb]))

Best Threshold=0.153012, F-Score=0.840, Precision=0.543, Recall=0.973


In [117]:
table = {'Metrics':['F-Score', 'Precision', 'Recall', 'Roc Auc'],\
         'LogReg':[f_LogReg[ix_lr], prec_LogReg[ix_lr], rec_LogReg[ix_lr], rocauc_LogReg],\
        'RandFor':[f_RandFor[ix_rf], prec_RandFor[ix_rf], rec_RandFor[ix_rf], rocauc_RandFor],\
        'GradBoost':[f_GradBoost[ix_gb], prec_GradBoost[ix_gb], rec_GradBoost[ix_gb],rocauc_RandFor]}

In [118]:
pd.pivot_table(pd.DataFrame(table),
               index=["Metrics"])

,GradBoost,LogReg,RandFor
Metrics,,,
F-Score,0.840276,0.838175,0.840547
Precision,0.543279,0.544331,0.536409
Recall,0.973295,0.968940,0.979370
Roc Auc,0.797000,0.785000,0.797000


Поскольку стояла задача определения наличия заболевания, то будем оценивать по полноте (более подробно выбор описан в комментарии выше)<br><br>
По всем метрикам все модели похожи. В среднем градиентный бустинг показал результат лучше, чем ЛогРег, но немного хуже, чем Лес (можно подрегулировать эти модели подбором гиперпараметров для достижения лучших результатов).<br><br>
Для анализа датасетов с дисбалансом лучше использовать PR-curve, чем ROC-curve. (Но хорошего объяснения я не нашла, буду признательна за доп материалы на эту тему)